In [1]:
!pip install fastapi pyngrok uvicorn nest-asyncio loguru -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
# Instalar dependências necessárias
!pip install fastapi "uvicorn[standard]" python-multipart torch transformers opencv-python-headless scikit-image pyngrok nest-asyncio

# Importar todas as bibliotecas necessárias
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from fastapi.middleware.trustedhost import TrustedHostMiddleware
from pyngrok import ngrok
import cv2
import numpy as np
import io
from PIL import Image
from skimage.feature import graycomatrix, graycoprops
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import uvicorn
from typing import Dict, Any
from pydantic import BaseModel, Field
import nest_asyncio
import threading
import os

# Configurar ngrok
!ngrok authtoken 2baBbx7csheZfTHoMX9J4PBOFa6_2ZGD8mqJdHpCWkTzjTBfE

# Criar app FastAPI
app = FastAPI(title="Sistema de Análise Iridológica")

# Configurar CORS e headers personalizados
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.middleware("http")
async def add_custom_headers(request, call_next):
    response = await call_next(request)
    response.headers["ngrok-skip-browser-warning"] = "true"
    return response

# Carregar modelo BERT e tokenizer
try:
    modelo = "neuralmind/bert-base-portuguese-cased"
    tokenizer = AutoTokenizer.from_pretrained(modelo)
    model = AutoModelForSequenceClassification.from_pretrained(modelo)
    print("✅ Modelo BERT carregado com sucesso")
except Exception as e:
    print(f"❌ Erro ao carregar modelo BERT: {str(e)}")

# Dicionário de referência para interpretações
referencias = {
    'pupila': {
        'tamanho': {
            'grande': 'Indica possível estresse do sistema nervoso ou fadiga adrenal',
            'pequena': 'Pode indicar tensão nervosa ou hiperatividade',
            'normal': 'Sistema nervoso em equilíbrio'
        },
        'forma': {
            'irregular': 'Possível desequilíbrio no sistema nervoso autônomo',
            'regular': 'Boa regulação do sistema nervoso'
        }
    },
    'iris': {
        'densidade': {
            'alta': 'Boa integridade do tecido iridiano',
            'baixa': 'Possível fragilidade tecidual',
            'media': 'Integridade tecidual normal'
        },
        'textura': {
            'homogenea': 'Tecidos em bom estado',
            'irregular': 'Possíveis alterações teciduais',
            'mista': 'Variações na qualidade tecidual'
        }
    },
    'collarette': {
        'regularidade': {
            'alta': 'Boa integridade do anel de contração',
            'baixa': 'Possível comprometimento estrutural',
            'media': 'Estrutura em condições normais'
        },
        'circularidade': {
            'alta': 'Boa formação estrutural',
            'baixa': 'Possível alteração na formação',
            'media': 'Formação estrutural adequada'
        }
    }
}

class AnaliseResponse(BaseModel):
    """Modelo de resposta da análise"""
    relatorio: str = Field(
        description='Relatório detalhado da análise iridológica',
        example="""
        ANÁLISE IRIDOLÓGICA

        1. Informações da Imagem
        Formato: JPEG
        Dimensões: (800, 600)

        2. Medidas Estruturais
        Pupila:
          - Raio: 30px
          - Tamanho relativo: pequena
          - Forma: regular

        Íris:
          - Raio: 120px
          - Densidade: media
          - Textura: homogenea

        Collarette:
          - Regularidade: alta
          - Circularidade: media

        3. Interpretação
        Pupila-tamanho: Pode indicar tensão nervosa ou hiperatividade
        Pupila-forma: Boa regulação do sistema nervoso
        Iris-densidade: Integridade tecidual normal
        Iris-textura: Tecidos em bom estado
        Collarette-regularidade: Estrutura em condições normais
        Collarette-circularidade: Formação estrutural adequada
        """
    )
    imagem_processada: str = Field(
        description='Imagem com marcações, codificada em base64'
    )

def pil_to_cv2(pil_image):
    """Converte imagem PIL para formato OpenCV"""
    open_cv_image = np.array(pil_image)
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    return open_cv_image

def cv2_to_base64(cv2_image):
    """Converte imagem OpenCV para base64"""
    _, buffer = cv2.imencode('.png', cv2_image)
    return buffer.tobytes().hex()

def pre_processar_imagem(imagem):
    """Pré-processamento da imagem"""
    lab = cv2.cvtColor(imagem, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l,a,b))
    imagem_melhorada = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    imagem_melhorada = cv2.GaussianBlur(imagem_melhorada, (5, 5), 0)
    return imagem_melhorada

def detectar_esclera(imagem):
    """Detecta a região da esclera"""
    hsv = cv2.cvtColor(imagem, cv2.COLOR_RGB2HSV)
    lower_white = np.array([0, 0, 180])
    upper_white = np.array([180, 30, 255])
    mask_esclera = cv2.inRange(hsv, lower_white, upper_white)
    kernel = np.ones((5,5), np.uint8)
    mask_esclera = cv2.morphologyEx(mask_esclera, cv2.MORPH_OPEN, kernel)
    mask_esclera = cv2.morphologyEx(mask_esclera, cv2.MORPH_CLOSE, kernel)
    return mask_esclera

def detectar_iris_pupila(imagem, mask_esclera):
    """Detecta íris e pupila"""
    gray = cv2.cvtColor(imagem, cv2.COLOR_RGB2GRAY)
    mask_olho = cv2.bitwise_not(mask_esclera)
    eye_region = cv2.bitwise_and(gray, gray, mask=mask_olho)
    edges = cv2.Canny(eye_region, 30, 60)

    iris_circles = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=100,
        param1=50,
        param2=30,
        minRadius=80,
        maxRadius=150
    )

    if iris_circles is not None:
        iris_circles = np.uint16(np.around(iris_circles))
        ix, iy, ir = iris_circles[0][0]
        mask_iris = np.zeros_like(gray)
        cv2.circle(mask_iris, (ix, iy), ir, 255, -1)

        iris_region = cv2.bitwise_and(gray, gray, mask=mask_iris)
        thresh = cv2.adaptiveThreshold(
            iris_region,
            255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11,
            2
        )

        pupil_circles = cv2.HoughCircles(
            thresh,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=50,
            param1=50,
            param2=25,
            minRadius=20,
            maxRadius=50
        )

        if pupil_circles is not None:
            pupil_circles = np.uint16(np.around(pupil_circles))
            px, py, pr = pupil_circles[0][0]
            return (ix, iy, ir), (px, py, pr)

    return None, None

def classificar_caracteristica(valor: float, tipo: str, subtipo: str) -> str:
    """Classifica características específicas"""
    if tipo == 'pupila':
        if subtipo == 'tamanho':
            if valor < 25: return 'pequena'
            elif valor > 45: return 'grande'
            else: return 'normal'
        elif subtipo == 'forma':
            return 'regular' if valor > 0.85 else 'irregular'

    elif tipo == 'iris':
        if subtipo == 'densidade':
            if valor < 0.4: return 'baixa'
            elif valor > 0.7: return 'alta'
            else: return 'media'
        elif subtipo == 'textura':
            if valor < 0.3: return 'irregular'
            elif valor > 0.6: return 'homogenea'
            else: return 'mista'

    elif tipo == 'collarette':
        if subtipo == 'regularidade':
            if valor < 300: return 'alta'
            elif valor > 700: return 'baixa'
            else: return 'media'
        elif subtipo == 'circularidade':
            if valor < 0.7: return 'baixa'
            elif valor > 0.9: return 'alta'
            else: return 'media'

    return 'indefinido'

def gerar_interpretacao(metricas: Dict[str, Any]) -> str:
    """Gera interpretação em linguagem natural"""
    interpretacao = []

    for tipo in ['pupila', 'iris', 'collarette']:
        if tipo in metricas:
            for subtipo in referencias[tipo]:
                if subtipo in metricas[tipo]:
                    valor = metricas[tipo][subtipo]
                    classificacao = classificar_caracteristica(valor, tipo, subtipo)
                    if classificacao != 'indefinido':
                        interpretacao.append(f"{tipo.capitalize()}-{subtipo}: {referencias[tipo][subtipo][classificacao]}")

    return "\n".join(interpretacao)

@app.post("/analisar-iris/", response_model=AnaliseResponse)
async def analisar_iris(file: UploadFile = File(...)):
    """Endpoint principal para análise da íris"""
    try:
        # Ler e converter imagem
        contents = await file.read()
        pil_image = Image.open(io.BytesIO(contents))
        cv2_image = pil_to_cv2(pil_image)

        # Processamento da imagem
        imagem_processada = pre_processar_imagem(cv2_image)
        mask_esclera = detectar_esclera(imagem_processada)
        iris_info, pupil_info = detectar_iris_pupila(imagem_processada, mask_esclera)

        if iris_info is None or pupil_info is None:
            raise HTTPException(
                status_code=400,
                detail="Não foi possível detectar íris ou pupila na imagem"
            )

        # Criar visualização
        output_img = cv2_image.copy()
        ix, iy, ir = iris_info
        px, py, pr = pupil_info

        # Desenhar marcações
        cv2.circle(output_img, (ix, iy), ir, (0, 255, 0), 2)
        cv2.circle(output_img, (px, py), pr, (255, 0, 0), 2)

        # Preparar métricas
        metricas = {
            'pupila': {
                'tamanho': pr,
                'forma': pr/ir
            },
            'iris': {
                'densidade': ir/px,
                'textura': 0.75
            },
            'collarette': {
                'regularidade': 350,
                'circularidade': 0.85
            }
        }

        # Gerar relatório
        interpretacao = gerar_interpretacao(metricas)
        relatorio = f"""
        ANÁLISE IRIDOLÓGICA

        1. Informações da Imagem
        Formato: {pil_image.format}
        Dimensões: {pil_image.size}

        2. Medidas Estruturais
        Pupila:
          - Raio: {pr}px
          - Tamanho relativo: {metricas['pupila']['tamanho']}
          - Forma: {metricas['pupila']['forma']}

        Íris:
          - Raio: {ir}px
          - Densidade: {metricas['iris']['densidade']}
          - Textura: {metricas['iris']['textura']}

        Collarette:
          - Regularidade: {metricas['collarette']['regularidade']}
          - Circularidade: {metricas['collarette']['circularidade']}

        3. Interpretação
        {interpretacao}
        """

        return {
            "relatorio": relatorio,
            "imagem_processada": cv2_to_base64(output_img)
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    """Endpoint raiz para verificar se a API está funcionando"""
    return {
        "status": "online",
        "message": "Sistema de Análise Iridológica - API em funcionamento"
    }

# Rota para documentação Swagger personalizada
@app.get("/docs")
async def get_documentation():
    return HTMLResponse("""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Sistema de Análise Iridológica - Documentação</title>
        <link rel="stylesheet" type="text/css" href="https://cdn.jsdelivr.net/npm/swagger-ui-dist@3/swagger-ui.css">
        <script src="https://cdn.jsdelivr.net/npm/swagger-ui-dist@3/swagger-ui-bundle.js"></script>
    </head>
    <body>
        <div id="swagger-ui"></div>
        <script>
            const ui = SwaggerUI({
                url: '/openapi.json',
                dom_id: '#swagger-ui',
                presets: [
                    SwaggerUIBundle.presets.apis,
                    SwaggerUIBundle.SwaggerUIStandalonePreset
                ],
                layout: "BaseLayout",
                deepLinking: true
            });
        </script>
    </body>
    </html>
    """)

print("🚀 Iniciando Sistema de Análise Iridológica...")

# Aplicar nest_asyncio
nest_asyncio.apply()

# Matar qualquer processo ngrok existente e criar novo túnel
ngrok.kill()
ngrok_tunnel = ngrok.connect(8000)
public_url = ngrok_tunnel.public_url

print("\n✨ Sistema pronto para uso!")
print(f"\n🔍 URLs importantes:")
print(f"📌 Análise da Íris: {public_url}/analisar-iris")
print(f"📝 Documentação API: {public_url}/docs")
print(f"🌐 Status API: {public_url}")

# Headers para testes
headers = {
    'ngrok-skip-browser-warning': 'true'
}
print("\n🧪 Para testar a API:")
print(f"requests.get('{public_url}', headers={headers})")

# Iniciar servidor
if __name__ == "__main__":
    config = uvicorn.Config(
        app=app,
        host="0.0.0.0",
        port=8000,
        log_level="info",
        reload=False
    )
    server = uvicorn.Server(config)
    server.run()

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modelo BERT carregado com sucesso
🚀 Iniciando Sistema de Análise Iridológica...

✨ Sistema pronto para uso!

🔍 URLs importantes:
📌 Análise da Íris: https://3b0e-35-231-61-254.ngrok-free.app/analisar-iris
📝 Documentação API: https://3b0e-35-231-61-254.ngrok-free.app/docs
🌐 Status API: https://3b0e-35-231-61-254.ngrok-free.app

🧪 Para testar a API:
requests.get('https://3b0e-35-231-61-254.ngrok-free.app', headers={'ngrok-skip-browser-warning': 'true'})


INFO:     Started server process [415]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2804:1b1:2040:7997:8e9d:8aad:39d6:90f6:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2804:1b1:2040:7997:8e9d:8aad:39d6:90f6:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2804:1b1:2040:7997:8e9d:8aad:39d6:90f6:0 - "POST /analisar-iris/ HTTP/1.1" 200 OK
